In [2]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from django.db.models import Count, Avg, F, Sum
import math

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from utils.utils import ars

qid = 1558
q = Query.objects.get(pk=qid)

In [152]:

p = Project.objects.get(title="NETs")
pdocs = Doc.objects.filter(query__project=p)

techs = Technology.objects.filter(project=p).exclude(name__in=["Bioenergy","Storage","Blue Carbon"])

techs = Technology.objects.filter(project=p).exclude(name__in=["Blue Carbon"])

tdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))

qtdocs = set(list(Doc.objects.filter(
    query__technology__in=techs,
    query__type="default"
).values_list('UT',flat=True)))

## These are all docs in the project

pdocs =  Doc.objects.filter(
    UT__in=tdocs.union(qtdocs)
)

print(pdocs.distinct().count())#techs = Technology.objects.filter(project=p)


5766


In [153]:
queries = Query.objects.filter(technology__in=techs,doc__isnull=False,type="default").distinct()

qts = queries.annotate(
    ndocs = Count('doc')
)

qtdf = pd.DataFrame.from_dict(list(qts.order_by('id').values('id','technology__name','ndocs','title','text','database')))

qtdf.head()

,database,id,ndocs,technology__name,text,title
0,WoS,243,198,Direct Air Capture,(TS = (((capture OR extraction OR absorbtion) ...,NETs_scoping_DAC
1,WoS,286,31,BECCS,TI = ((potential* OR supply) AND (bioenergy* O...,test
2,WoS,303,87,Bioenergy,TI = ((potential* OR supply OR production OR s...,BioEn Potential WoS
3,WoS,330,660,Soil Carbon Sequestration,TS = ((soil NEAR/3 (carbon OR CO2) NEAR/3 (seq...,soil_carbon_sequestration
4,scopus,332,637,Soil Carbon Sequestration,TITLE-ABS-KEY ( ( soil W/3 ( carbon OR co2...,soil_carbon_sequestration_scopus


In [154]:
trdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=techs
).values_list('UT',flat=True)))

ttdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))


allreldocs =  Doc.objects.filter(
    UT__in=trdocs.union(ttdocs)
)

print(allreldocs.count())

1997


In [155]:
global unfound
unfound = 0

global multmatches
multmatches = 0

global found
found = 0

global indb
indb = 0

spapers = pd.read_csv('../tables/unique_spreadsheet_papers.csv')


def find_paper(x):
    
    try:
    
        tis = x['X3'].split()
        
    except:
        print(x['X3'])
        tis = str(x['X3']).split()
    
    if len(tis) > 8:
        ti = ' '.join(tis[:8])
    else:
        ti = x['X3']
        
    docs = pdocs.filter(title__icontains=ti)
    
    global unfound
    global found
    global multmatches
    
    if docs.count() == 1:
    
        doc = docs.first()
        found +=1
    elif docs.count() > 1:
        doc = "None"
        multmatches+=1
    else:       
        di = x['X3'].replace('http://dx.doi.org/','')
        docs = pdocs.filter(wosarticle__di=di)
        if docs.count()==1:
            doc = docs.first()
            found+=1
        elif docs.count() > 1:
            doc = "None"
            multmatches+=1
        else:
            au = x['X1'].split()[0].split(',')[0]
            docs = pdocs.filter(docauthinst__AU__icontains=au,docauthinst__position=1,PY=x['X2']).distinct()
            if docs.count()==1:
                doc = docs.first()
                found+=1
            elif docs.count() > 1:
                doc = "None"
                multmatches+=1
            else:
                docs = pdocs.filter(docauthinst__AU__icontains=au,docauthinst__position__in=[1,0],PY=x['X2']).distinct()
                if docs.count()==1:
                    doc = docs.first()
                    found+=1
                else:
                    ti = re.findall('[A-Z]{1}\.{1}[^;]([^\.]{5,})',x['X3'])
                    if len(ti) > 0:
                        ti = ti[0]
                    else:
                        ti = "zwasdfölkjag"
                    docs = pdocs.filter(title__icontains=ti)
                    if docs.count()==1:
                        doc = docs.first()
                        found+=1
                    else:
                        doc = "None"
                        unfound+=1
                        
    try:
        doc = doc.UT
    except:
        doc = doc
    
    return doc

spapers['doc_id'] = spapers.apply(lambda x: find_paper(x), axis=1)

print("no matches: {}".format(unfound))
print("multiple matches: {}".format(multmatches))
print("found {}".format(found))


spapers.head()

nan
no matches: 44
multiple matches: 10
found 807


,Unnamed: 0,X1,X2,X3,technology,doc_id
0,1,"Renforth, P",2012.0,The potential of enhanced weathering in the UK,Enhanced weathering (terrestrial and ocean),WOS:000309790500021
1,3,Schuiling & Krijgsman,2006.0,ENHANCED WEATHERING: AN EFFECTIVE AND CHEAP TO...,Enhanced weathering (terrestrial and ocean),WOS:000237646500016
2,4,Köhler et al.,2010.0,Geoengineering potential of artificially enhan...,Enhanced weathering (terrestrial and ocean),WOS:000284529000019
3,5,Taylor et al.,2016.0,Enhanced weathering strategies for stabilizing...,Enhanced weathering (terrestrial and ocean),WOS:000373060000019
4,6,Hangx & Spiers,2009.0,Coastal spreading of olivine to control atmosp...,Enhanced weathering (terrestrial and ocean),WOS:000272373000010


In [156]:
spapers[spapers['doc_id'] == "None"].reset_index()['X3']


notfound = spapers[spapers['doc_id'] == "None"].reset_index()

foundpapers = spapers[spapers['doc_id'] != "None"].reset_index()

notfound



#notfound.to_excel('../tables/spreadsheet_not_found.xlsx')


#spapers[spapers['doc_id'] == "None"].groupby('technology').count()

# Go through docs, mark include1 if in allreldocs, mark include2 if in list generated in R


#spapers[spapers['doc_id'] == "None"].reset_index()['X3'][18]




,index,Unnamed: 0,X1,X2,X3,technology,doc_id
0,132,136,"Xu, B, Guo, ZD, Piao, SL, Fang, JY",2010.0,Biomass carbon stocks in China’s forests betwe...,Afforestation and Reforestation,None
1,134,138,"Gorte, R.W.",2010.0,Carbon sequestration in forests,Afforestation and Reforestation,None
2,150,154,"Henry, M, Tittonell, P, Manlay, RJ, Bernoux, M...",2009.0,"Biodiversity, carbon stocks and sequestration ...",Afforestation and Reforestation,None
3,196,202,"Glenday, J",2006.0,Carbon storage and emissions offset potential ...,Afforestation and Reforestation,None
4,213,219,Melvin G.R. Cannell,2003.0,Carbon sequestration and biomass energy offset...,Afforestation and Reforestation,None
5,214,222,"Sohngen, B, Mendelsohn, R",2003.0,An Optimal Control Model of Forest Carbon Sequ...,Afforestation and Reforestation,None
6,223,232,Stavins and Richards,2002.0,The cost of U.S. forest-based carbon sequestra...,Afforestation and Reforestation,None
7,239,248,Kaiser,2000.0,Panel Estimates Possible Carbon 'Sinks',Afforestation and Reforestation,None
8,249,258,Brown,1996.0,Management of forests for mitigation of greenh...,Afforestation and Reforestation,None
9,272,282,NaN,NaN,NaN,BECCS,None


In [129]:
print('WOS:000309790500021' in foundpapers['doc_id'])

foundpapers['doc_id'].tolist()

False


['WOS:000309790500021',
 'WOS:000237646500016',
 'WOS:000284529000019',
 'WOS:000373060000019',
 'WOS:000272373000010',
 'WOS:000307378500011',
 'WOS:000398646500042',
 'WOS:000260293100007',
 'WOS:000371488300009',
 'WOS:000316998300017',
 'WOS:000313220300016',
 'WOS:000288146200002',
 'WOS:000287106700015',
 'WOS:000264867000005',
 'WOS:000251582800043',
 'WOS:000168824703328',
 'WOS:000323230100001',
 '2-s2.0-84978123737',
 '2-s2.0-84988701357',
 'WOS:000389184600020',
 'WOS:000392684700013',
 '2-s2.0-84986325555',
 '2-s2.0-84992168841',
 '2-s2.0-84979681368',
 'WOS:000376713100009',
 'WOS:000378949300001',
 'WOS:000380910100076',
 'WOS:000381381100071',
 'WOS:000381828300018',
 'WOS:000381837900016',
 'WOS:000384103400003',
 'WOS:000385598800030',
 'WOS:000388307100015',
 'WOS:000389089300002',
 'WOS:000389961800016',
 'WOS:000391425900009',
 'WOS:000369555700013',
 'WOS:000370887100016',
 'WOS:000371678700007',
 'WOS:000370896400006',
 'WOS:000367755700044',
 'WOS:000358268700005

In [ ]:
pdoc_dict = []

allrelids = list(allreldocs.values_list('UT',flat=True))

for doc in pdocs:
    das = doc.docauthinst_set.order_by('AU','position').distinct('AU').values_list('id',flat=True)
    unique = doc.docauthinst_set.filter(id__in=das).order_by('position').values_list('AU',flat=True)
    qset = [str(x) for x in list(doc.query.filter(
            technology__in=techs,type="default"
        ).distinct().values_list('id',flat=True))]
    try:
        py = int(doc.PY)
    except:
        py = doc.PY
    if doc.UT in foundpapers['doc_id'].tolist():
        sp = 1
    else:
        sp = 0
    if doc.UT in allrelids:
        r = 1
        dtechs = techs.filter(query__docownership__doc=doc,query__docownership__relevant=1) | techs.filter(doc=doc)
        #dtechs = doc.docownership_set
        dtechs = [str(x.name) for x in dtechs.distinct()]
    else:
        r = 0
        dtechs = []
        
        
    pdoc = {
        'Authors': "; ".join(unique),
        'Publication_Year': py,
        'Title': doc.title,
        'Queries': "; ".join(qset),
        'Relevant': r,
        'In_Spreadsheet': sp,
        'Technologies': '; '.join(dtechs)
    }
    pdoc_dict.append(pdoc)
    

In [146]:
doc = 'WOS:000353073400006'

dtechs = techs.filter(query__docownership__doc=doc,query__docownership__relevant=1) | techs.filter(doc=doc)

dtechs.distinct()

<QuerySet [<Technology: Afforestation/reforestation>]>

In [150]:
big_df = pd.DataFrame.from_dict(pdoc_dict)

big_df.head()

,Authors,In_Spreadsheet,Publication_Year,Queries,Relevant,Technologies,Title
0,"Mason, S.S.",0,1993.0,374; 672; 896,0,,Syntheses and 13C NMR Investigations of an Ext...
1,"Englert, R.",0,1991.0,369; 468; 661; 662; 1481,0,,Automated trace analysis of airborne C1- and C...
2,"Alig, R.",1,2000.0,334; 618,1,Afforestation/reforestation,"Mitigation, adaptation, and climate change: Re..."
3,"Dubey, M.K.; Ziock, H.; Rueff, G.; Elliott, S....",0,2002.0,369; 468; 661; 662; 1481,1,Direct Air Capture,Extraction of carbon dioxide from the atmosphe...
4,"MacLaren, P.",1,1999.0,334; 618,1,Afforestation/reforestation,Modelling the effect of land-use practices on ...


In [132]:
big_df.groupby(['In_Spreadsheet','Relevant']).count()

Authors  Publication_Year  Queries  Title
In_Spreadsheet Relevant                                           
0              0            5571              5571     5571   5571
               1            1239              1238     1239   1238
1              0              36                36       36     36
               1             758               758      758    758

In [151]:
big_df.to_excel('../tables/all_considered_docs.xlsx')

qtdf.to_excel('../tables/all_queries.xlsx')

In [33]:
for q in queries[:10]:
    ds = q.doc_set.all()
    dids = list(q.doc_set.all().values_list('UT',flat=True))
    ds = Doc.objects.filter(UT__in=dids)
    print(ds.count())
    ds = ds.annotate(
        qs = Count('query')
    )
    print(ds.filter(qs__lt=4).count())
    
    print(q.title)

68
0
enhanced_weathering_wos
2
0
forests1_backward_1_1
3
0
*7B_BlueDemos
101
0
enhanced_weathering_wos_2
637
0
soil_carbon_sequestration_scopus
4
0
DAC_extra
422
9
synonyms_all
660
0
soil_carbon_sequestration
396
0
beccs_scopus
391
6
synonyms_scopus


In [25]:
q = queries.first()

ds = q.doc_set.all()

print(ds.count())

ds = ds.annotate(
    qs = Count('query')
)

print(ds.filter(qs__lt=2).count())

q.title

198
198


'NETs_scoping_DAC'

In [ ]:


trdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=techs
).values_list('UT',flat=True)))

ttdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))

print(len(trdocs))
print(len(ttdocs))

allreldocs =  Doc.objects.filter(
    UT__in=trdocs.union(ttdocs)
)

print(allreldocs.count())

print(allreldocs.order_by('PY').first().PY)
print(allreldocs.order_by('PY').first().title)

print(allreldocs.filter(PY__lt=1990).count())